In [5]:
import logmmse
import wave
import logging

import sox
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

import librosa

import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaners import japanese_cleaners2
from text import cleaned_text_to_sequence
from pypinyin import lazy_pinyin, Style
import os
from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = cleaned_text_to_sequence(text)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm


# hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")
# hps = utils.get_hparams_from_file("./configs/rmzk_base.json")
hps = utils.get_hparams_from_file("./configs/mmj.json")
# net_g_ms = SynthesizerTrn(
#     len(symbols),
#     hps_ms.data.filter_length // 2 + 1,
#     hps_ms.train.segment_size // hps.data.hop_length,
#     n_speakers=hps_ms.data.n_speakers,
#     **hps_ms.model)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

net_g_ms = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g_ms.eval()




def tts(text):
    stn_tst = get_text(text, hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][
            0, 0].data.float().numpy()
        filename = 'file/out1.wav'
        write(filename, 22050, audio)
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))


def jtts(text,length_scale):

    # tokenization
    stn_tst = get_text(japanese_cleaners2(text), hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=length_scale)[0][
            0, 0].data.float().numpy()
        filename = 'resultwav/' +  text.replace('?','') + '.wav'
        sf.write(filename,  audio,22050)
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))


    # path = 'resultwav/' + text + '.wav'
    # f = wave.open(path, "r")
    # params = f.getparams()
    # nchannels, sampwidth, framerate, nframes = params[:4]
    # print("nchannels:", nchannels, "sampwidth:", sampwidth, "framerate:", framerate, "nframes:", nframes)
    # data = f.readframes(nframes)
    # f.close()
    # data = np.fromstring(data, dtype=np.short)
    #
    # # 降噪
    # data = logmmse.logmmse(data=data, sampling_rate=framerate)
    # # 保存音频
    # file_save = 'resultwav/' +'降噪_'+ text + '.wav'
    # nframes = len(data)
    # f = wave.open(file_save, 'w')
    # f.setparams((1, 2, framerate, nframes, 'NONE', 'NONE'))  # 声道，字节数，采样频率，*，*
    # # print(data)
    # f.writeframes(data)  # outData
    # f.close()



In [7]:
_ = utils.load_checkpoint("models/G_8000.pth", net_g, None)
# _ = utils.load_checkpoint("v_mzk/G_78000.pth", net_g, None)

INFO:root:Loaded checkpoint 'models/G_8000.pth' (iteration 21)


In [9]:
def multi_tts(text,speaker_id):
    # speaker_id = "0" #@param [0, 1, 2, 3, 4]
    speaker_id = int(speaker_id)
    length_scale = 1.1 #@param {type:"slider", min:0.1, max:3, step:0.05}
    sid = torch.LongTensor([speaker_id])
    stn_tst = get_text(text, hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=length_scale)[0][0,0].data.float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

In [12]:
multi_tts("そうと決まれば" ,0)